### A. Librerias

In [58]:
import os
import time
import json
import requests

import pandas as pd

from utils import sheets
from bs4 import BeautifulSoup

### B. Funciones

In [2]:
def get_files(path, extension):
    dir_name  = os.path.join(os.getcwd(), path)
    files     = os.listdir(dir_name)
    
    paths    = []
    for file in files:
        if extension in file:
            paths.append(os.path.join(path,file))

    return paths

def clean_files(path, extension='.json'):
    try:
        files = get_files(path, extension)
        for file in files:
            os.remove(file)
        print('Se eliminaron los archivos de la carpeta')
    except:
        pass

def generar_url_google_maps(latitud, longitud):
    url = f"https://www.google.com/maps?q={latitud},{longitud}"
    return url

def generar_url_street_view(latitud, longitud):
    url = f"https://www.google.com/maps?q=&layer=c&cbll={latitud},{longitud}"
    return url

### C. Ejecución - Buscar departamentos por distritos

In [3]:
# 1. Listar los distritos y sus urls
distritos_urls = [
                  ['surquillo','https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-lotes-o-casas-en-surquillo-lima-lima-150141']
                , ['miraflores','https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-lotes-o-casas-en-miraflores-lima-lima-150122']
                , ['barranco','https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-barranco-lima-lima-150104']
                , ['breña','https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-bre%C3%B1a-lima-lima-150105']
                , ['jesus_maria', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-jesus-maria-lima-lima-150113']
                , ['lince', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-lince-lima-lima-150116']
                , ['magdalena', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-magdalena-del-mar-lima-lima-150120']
                , ['pueblo_libre', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-pueblo-libre-lima-lima-150121']
                , ['san_isidro', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-san-isidro-lima-lima-150131']
                , ['san_miguel', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-san-miguel-lima-lima-150136']
                , ['la_victoria', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-san-miguel-lima-lima-150136']
            ]

In [4]:
# 2. Extraer los datos de cada distrito
for distrito, url in distritos_urls:
    # 1. Obtener la URL por request
    response = requests.get(url)
    print(response.status_code, '-> ', distrito)

    if response.status_code == 200:
        # 2. Parsear el HTML
        html = response.text

        # 3. Guardar el HTML en un archivo
        with open(os.path.join(os.getcwd(),'originals', distrito + '.html'), 'w') as outfile:
            outfile.write(html)

        # 4. Tiempo de espera
        time.sleep(2)

surquillo 200
miraflores 200
barranco 200
breña 200
jesus_maria 200
lince 200
magdalena 200
pueblo_libre 200
san_isidro 200
san_miguel 200
la_victoria 200


In [5]:
# 3. Listar los proyectos y sus urls
urls_list = []
for distrito, x in distritos_urls:
    # A. Leer el html
    with open(os.path.join(os.getcwd(),'originals', distrito+ '.html'), 'r') as infile:
        lines = infile.readlines()

    # B. Buscar la línea que contiene la palabra 'url'
    cadena = ''
    for i in range(len(lines)-1,0,-1):
        if '"url":' in lines[i]:
            cadena = lines[i]
            break
    
    # C. Buscar las urls
    posiciones_url = []
    posicion = cadena.find('"url":')

    while posicion != -1:
        posiciones_url.append(posicion)
        posicion = cadena.find('"url":', posicion + 1)
    
    # D. Buscar los project_id
    posiciones_ids = []
    posicion = cadena.find('"project_id":')

    while posicion != -1:
        posiciones_ids.append(posicion)
        posicion = cadena.find('"project_id":', posicion + 1)

    # E. Buscar links
    for a,b in zip(posiciones_url, posiciones_ids):
        url = ''.join(list(cadena)[a + 7 :b - 2])
        url = url.replace('\x5C','')  
        url = str(url)

        base = 'https://nexoinmobiliario.pe/proyecto/venta-de-departamento-' + url.split('/')[-1]
        urls_list.append(base)

len(urls_list)
    

481

In [6]:
# 4. Extraer los datos de cada proyecto
modelos_list = []
dataframes_list = []
for url in urls_list[:10]:
    # A. Obtener la URL por request
    response = requests.get(url)
    # print(response.status_code,':' ,url)

    if response.status_code == 200:
        # Parsea el contenido HTML de la página
        soup = BeautifulSoup(response.text, 'html.parser')

        #### A. Datos generales
        # Extrae el texto de <h1>Eres Fase 2</h1>
        proyecto = soup.find('h1').get_text(strip=True)
        # print(f'Proyecto: {proyecto}')

        # Extrae el texto de <p class="Project-header-address">Av. República de Panamá 4077</p>
        direccion = soup.find('p', class_='Project-header-address').get_text(strip=True)
        # print(f'Dirección: {direccion}')

        # Extrae el texto de <p class="Project-header-address"> - Surquillo</p>
        distrito = soup.find('p', class_='Project-header-address').find_next_sibling('p').get_text(strip=True).replace('-', '').strip()
        # print(f'Distrito: {distrito}')

        # Extrae el precio desde <p class="Project-header-price"><span class="title-price">Precio desde</span> <strong> S/. 319,118</strong></p>
        precio_desde = soup.find('p', class_='Project-header-price').strong.get_text(strip=True)
        # print(f'Precio desde: {precio_desde}')

        # Extrae el valor de referencia <p class="title-price">(<span>valor referencia </span><strong>$ 84,940</strong>)</p>
        valor_referencia = soup.find('p', class_='title-price').strong.get_text(strip=True)
        # print(f'Valor de referencia: {valor_referencia}')

        # Listar las áreas comunes
        areas_lista = soup.find('ul', class_='Project-areas-list')
        elementos_lista = areas_lista.find_all('li')
        areas_comunes = ', '.join(elemento.get_text(strip=True) for elemento in elementos_lista)

        # Encuentra el campo de entrada para la latitud
        input_latitud = soup.find('input', id='latitude')
        latitud = input_latitud['value']
        # print(f'Latitud: {latitud}')

        input_longitud = soup.find('input', id='longitude')
        longitud = input_longitud['value']
        # print(f'Longitud: {longitud}')

        # print('')
        # print('Datos de la inmobiliaria')
        #### B. Datos de la inmobiliaria
        # Encuentra el div con la clase "bx-data-project box-st"
        div_informacion_proyecto = soup.find('div', class_='bx-data-project box-st')

        if div_informacion_proyecto:
            # Encuentra todas las filas de la tabla dentro del div
            filas_tabla = div_informacion_proyecto.find('table').find_all('tr')

            # Itera sobre las filas de la tabla e imprime la información
            for fila in filas_tabla:
                columnas = fila.find_all('td')
                if len(columnas) == 2:
                    try:
                        etiqueta = columnas[0].strong.get_text(strip=True)
                        valor = columnas[1].get_text(strip=True)
                        
                        if etiqueta == 'Tipo de inmueble':
                            tipo_inmueble = valor
                        elif etiqueta == 'Área total':
                            area_total = valor
                        elif etiqueta == 'Dormitorios':
                            dormitorios = valor
                        elif etiqueta == 'Etapa del proyecto':
                            etapa_proyecto = valor
                        elif etiqueta == 'Fecha de entrega':
                            fecha_entrega = valor
                        elif etiqueta == 'Financiamiento':
                            financiamiento = valor
                        elif etiqueta == 'Inmobiliaria Nombre':
                            inmobiliaria_nombre = valor
                        
                        # print(f'{etiqueta} -> {valor}')

                    except:
                        pass

            # También puedes extraer información específica usando el método find para elementos específicos
            # Por ejemplo, para obtener la información de la inmobiliaria
            inmobiliaria_nombre = div_informacion_proyecto.find('tr', class_='Project-inmobiliaria').find('h2').get_text(strip=True)
            
            # print(f'Inmobiliaria Nombre: {inmobiliaria_nombre}')

        #### C. Modelos disponibles
        modelos_disponibles = soup.find_all('div', class_='Project-available-model')
        modelos_list.append([proyecto,modelos_disponibles, url])
        # print('')
        # print('Modelos disponibles')
        # print(f'Cantidad de modelos: {len(modelos_disponibles)}')

        #### D. Unir los datos en un diccionario
        proyecto_data = {
            'Proyecto': proyecto,
            'Dirección': direccion,
            'Distrito': distrito,
            'Precio Desde': precio_desde,
            'Valor de Referencia': valor_referencia,
            'Latitud': latitud,
            'Longitud': longitud,
            'Tipo de Inmueble': tipo_inmueble,
            'Área Total': area_total,
            'Dormitorios': dormitorios,
            'Etapa del Proyecto': etapa_proyecto,
            'Fecha de Entrega': fecha_entrega,
            'Financiamiento': financiamiento,
            'Inmobiliaria Nombre': inmobiliaria_nombre,
            'Áreas Comunes': areas_comunes,
            'URL Nexoinmobiliario': url
        }

        # Convierte el diccionario en un DataFrame
        df = pd.DataFrame([proyecto_data])
        dataframes_list.append(df)

        # Tiempo de espera
        time.sleep(2)

In [45]:
# 1. Concatenar los DataFrames
df_proyectos = pd.concat(dataframes_list)

# 2. Crear columnas de Google Maps y Street View
df_proyectos['Google Maps'] = df_proyectos.apply(lambda row: generar_url_google_maps(row['Latitud'], row['Longitud']), axis=1)
df_proyectos['Street View'] = df_proyectos.apply(lambda row: generar_url_street_view(row['Latitud'], row['Longitud']), axis=1)

# Eliminar las columnas de Latitud y Longitud
df_proyectos = df_proyectos.drop(columns=['Latitud', 'Longitud'])
df_proyectos.head(3)

,Proyecto,Dirección,Distrito,Precio Desde,Valor de Referencia,Tipo de Inmueble,Área Total,Dormitorios,Etapa del Proyecto,Fecha de Entrega,Financiamiento,Inmobiliaria Nombre,Áreas Comunes,URL Nexoinmobiliario,Google Maps,Street View
0,Eres Fase 2,Av. República de Panamá 4077,Surquillo,"S/. 319,118","$ 84,940",Departamento,41.18 a 80.59 m2,1 a 3,En planos,"30 de Septiembre, 2025",Banco BCP,GRUPO LAR,"Área de juegos para niños, Jardin de niños, Lo...",https://nexoinmobiliario.pe/proyecto/venta-de-...,"https://www.google.com/maps?q=-12.1040361,-77....",https://www.google.com/maps?q=&layer=c&cbll=-1...
0,CAPRI,EL CEREZO 274,La Calera De La Merced Surquillo,"S/. 370,000","$ 98,483",Departamento,50.00 a 81.00 m2,2 a 3,En construcción,"29 de Febrero, 2024",Financiamiento Propio,INMOBILIARIA CANASAN,Lobby,https://nexoinmobiliario.pe/proyecto/venta-de-...,https://www.google.com/maps?q=-12.117575316211...,https://www.google.com/maps?q=&layer=c&cbll=-1...
0,DORUE,"Av. Domingo Orué 230, Surquillo",Surquillo,"S/. 245,000","$ 65,212",Departamento,35.08 a 69.34 m2,1 a 2,En planos,"31 de Diciembre, 2025",Financiamiento Propio,DESARROLLADORA,"Areas verdes, Gimnasio, Lobby, Parque privado,...",https://nexoinmobiliario.pe/proyecto/venta-de-...,"https://www.google.com/maps?q=-12.108786,-77.0...",https://www.google.com/maps?q=&layer=c&cbll=-1...


### D. Ejecución - Buscar detalle de departamentos

In [46]:
data_modelos = []
for proyecto, modelos_disponibles, url in modelos_list:
    for modelo in modelos_disponibles:
        try:
            # Extrae información del enlace de imagen
            #imagen_enlace = modelo.find('a', class_='popup-link')['data-links'].strip()
            #print(f'Enlace de la imagen: {imagen_enlace}')

            # Extrae información del modelo
            nombre_modelo = modelo.find('span', class_='name_tipology').get_text(strip=True).strip()
            #print(f'Nombre del modelo: {nombre_modelo}')

            # Extrae información del número de pisos
            num_pisos = modelo.find('span', class_='num_pisos').get_text(strip=True).strip()
            #print(f'Número de pisos: {num_pisos}')

            # Extrae información del dormitorio, área y precio
            dormitorio = modelo.find('span', class_='bedroom').get_text(strip=True).strip()
            area = modelo.find('span', class_='area').get_text(strip=True).strip()
            precio = modelo.find('span', class_='price').get_text(strip=True).strip()

            #print(f'Dormitorio: {dormitorio}')
            #print(f'Área: {area}')
            #print(f'Precio desde: {precio}')

            # También puedes extraer información del enlace de cotización
            #enlace_cotizacion = modelo.find('a', class_='Btn-ver--red')['href'].strip()
            #print(f'Enlace de cotización: {enlace_cotizacion}')

            # Agregar datos del modelo a la lista
            data_modelos.append({
                'Proyecto': proyecto,
                'Nombre del modelo': nombre_modelo,
                'Número de pisos': num_pisos,
                'Dormitorio': dormitorio,
                'Área': area,
                'Precio desde': precio,
                'URL Nexoinmobiliario': url
            })

            #print("\n---\n")  # Separador entre modelos
        except:
            pass

df_modelos = pd.DataFrame(data_modelos)
df_modelos.head(3)

,Proyecto,Nombre del modelo,Número de pisos,Dormitorio,Área,Precio desde,URL Nexoinmobiliario
0,Eres Fase 2,A1,Pisos: Entre 2 al 9,1,52.55 ...,"S/ 442,532",https://nexoinmobiliario.pe/proyecto/venta-de-...
1,Eres Fase 2,A2,Pisos: Entre 2 al 11,2,67.64 ...,"S/ 532,915",https://nexoinmobiliario.pe/proyecto/venta-de-...
2,Eres Fase 2,A3,Pisos: Entre 2 al 8,1,50.67 ...,"S/ 430,756",https://nexoinmobiliario.pe/proyecto/venta-de-...


### E. Dar formato a df_proyectos

In [33]:
def formato_fecha(mes_str):
    meses = {
        'Enero': '01',
        'Febrero': '02',
        'Marzo': '03',
        'Abril': '04',
        'Mayo': '05',
        'Junio': '06',
        'Julio': '07',
        'Agosto': '08',
        'Septiembre': '09',
        'Octubre': '10',
        'Noviembre': '11',
        'Diciembre': '12'
    }

    mes_split = mes_str.split(' ')

    dia = mes_split[0]
    mes = meses.get(mes_split[2].replace(',',''))
    anio = mes_split[3]

    mes_format = f'{anio}-{mes}-{dia}'
    if mes_format:
        return mes_format
    else:
        return None

In [94]:
# 1. Dar formato a las columnas de fecha y castear a date 'yyyy-mm-dd'
df_proyectos_format = df_proyectos.copy()
df_proyectos_format['Fecha de Entrega'] = df_proyectos_format['Fecha de Entrega'].apply(formato_fecha)
df_proyectos_format['Fecha de Entrega'] = pd.to_datetime(df_proyectos_format['Fecha de Entrega'])

# 2. Replace a las columnas de 'Precio Desde' .replace('S/.','').replace(',','') y 'Valor de Referencia' .replace('$','').replace(',','')
df_proyectos_format['Precio Desde'] = df_proyectos_format['Precio Desde'].str.replace('S/. ','').str.replace(',','')
df_proyectos_format['Valor de Referencia'] = df_proyectos_format['Valor de Referencia'].str.replace('$','').str.replace(',','')

# 3. Castear las columnas de 'Precio Desde' y 'Valor de Referencia' a int
df_proyectos_format['Precio Desde'] = df_proyectos_format['Precio Desde'].astype(int)
df_proyectos_format['Valor de Referencia'] = df_proyectos_format['Valor de Referencia'].astype(int)

# 4. Renombrar columna 'Precio Desde':'Precio Desde (S/)' , 'Valor de Referencia':'Valor de Referencia ($)'
df_proyectos_format = df_proyectos_format.rename(columns={'Precio Desde':'Precio Desde (S/.)', 'Valor de Referencia':'Valor de Referencia ($)'})

# 5. Hacer un upper a las columas que son string
df_proyectos_format['Proyecto'] = df_proyectos_format['Proyecto'].str.upper()
df_proyectos_format['Dirección'] = df_proyectos_format['Dirección'].str.upper()
df_proyectos_format['Financiamiento'] = df_proyectos_format['Financiamiento'].str.upper()
df_proyectos_format['Inmobiliaria Nombre'] = df_proyectos_format['Inmobiliaria Nombre'].str.upper()

# 6. Reordenar las columnas
df_proyectos_format = df_proyectos_format[['Proyecto', 'Distrito', 'Precio Desde (S/.)', 'Valor de Referencia ($)', 'Tipo de Inmueble', 'Área Total', 'Dormitorios', 'Etapa del Proyecto', 'Fecha de Entrega', 'Financiamiento', 'Inmobiliaria Nombre', 'Áreas Comunes', 'Dirección', 'Google Maps', 'Street View', 'URL Nexoinmobiliario']]

df_proyectos_format.head(3)

,Proyecto,Distrito,Precio Desde (S/.),Valor de Referencia ($),Tipo de Inmueble,Área Total,Dormitorios,Etapa del Proyecto,Fecha de Entrega,Financiamiento,Inmobiliaria Nombre,Áreas Comunes,Dirección,Google Maps,Street View,URL Nexoinmobiliario
0,ERES FASE 2,Surquillo,319118,84940,Departamento,41.18 a 80.59 m2,1 a 3,En planos,2025-09-30,BANCO BCP,GRUPO LAR,"Área de juegos para niños, Jardin de niños, Lo...",AV. REPÚBLICA DE PANAMÁ 4077,"https://www.google.com/maps?q=-12.1040361,-77....",https://www.google.com/maps?q=&layer=c&cbll=-1...,https://nexoinmobiliario.pe/proyecto/venta-de-...
0,CAPRI,La Calera De La Merced Surquillo,370000,98483,Departamento,50.00 a 81.00 m2,2 a 3,En construcción,2024-02-29,FINANCIAMIENTO PROPIO,INMOBILIARIA CANASAN,Lobby,EL CEREZO 274,https://www.google.com/maps?q=-12.117575316211...,https://www.google.com/maps?q=&layer=c&cbll=-1...,https://nexoinmobiliario.pe/proyecto/venta-de-...
0,DORUE,Surquillo,245000,65212,Departamento,35.08 a 69.34 m2,1 a 2,En planos,2025-12-31,FINANCIAMIENTO PROPIO,DESARROLLADORA,"Areas verdes, Gimnasio, Lobby, Parque privado,...","AV. DOMINGO ORUÉ 230, SURQUILLO","https://www.google.com/maps?q=-12.108786,-77.0...",https://www.google.com/maps?q=&layer=c&cbll=-1...,https://nexoinmobiliario.pe/proyecto/venta-de-...


In [95]:
df_proyectos_format.dtypes

Proyecto                           object
Distrito                           object
Precio Desde (S/.)                  int64
Valor de Referencia ($)             int64
Tipo de Inmueble                   object
Área Total                         object
Dormitorios                        object
Etapa del Proyecto                 object
Fecha de Entrega           datetime64[ns]
Financiamiento                     object
Inmobiliaria Nombre                object
Áreas Comunes                      object
Dirección                          object
Google Maps                        object
Street View                        object
URL Nexoinmobiliario               object
dtype: object

### F. Dar formato a df_modelos

In [123]:
# 1. Dar formato a las columnas de fecha y castear a date 'yyyy-mm-dd'
df_modelos_format = df_modelos.copy()

# 2. Replace a las columnas de 'Precio Desde' .replace('S/.','').replace(',','') y 'Valor de Referencia' .replace('$','').replace(',','')
df_modelos_format['Precio desde'] = df_modelos_format['Precio desde'].str.replace('S/ ','').str.replace(',','')
df_modelos_format['Precio desde'] = df_modelos_format['Precio desde'].astype(int)

# 3. Hacer un upper a las columas que son string
df_modelos_format['Proyecto'] = df_modelos_format['Proyecto'].str.upper()
df_modelos_format['Nombre del modelo'] = df_modelos_format['Nombre del modelo'].str.upper()

# 4. Hacer .split(' ')[0] a la columna 'Área'
df_modelos_format['Área'] = df_modelos_format['Área'].apply(lambda x: x.split(' ')[0])

# 5. Renombrar columna 'Número de pisos':'Piso del departamento', 'Área':'Área (m2)', 'Dormitorio':'Dormitorios'
df_modelos_format = df_modelos_format.rename(columns={'Número de pisos':'Piso del departamento', 'Área':'Área (m2)', 'Dormitorio':'Dormitorios'})

df_modelos_format.head(3)

,Proyecto,Nombre del modelo,Piso del departamento,Dormitorios,Área (m2),Precio desde,URL Nexoinmobiliario
0,ERES FASE 2,A1,Pisos: Entre 2 al 9,1,52.55,442532,https://nexoinmobiliario.pe/proyecto/venta-de-...
1,ERES FASE 2,A2,Pisos: Entre 2 al 11,2,67.64,532915,https://nexoinmobiliario.pe/proyecto/venta-de-...
2,ERES FASE 2,A3,Pisos: Entre 2 al 8,1,50.67,430756,https://nexoinmobiliario.pe/proyecto/venta-de-...


In [124]:
df_modelos_format.dtypes

Proyecto                 object
Nombre del modelo        object
Piso del departamento    object
Dormitorios              object
Área (m2)                object
Precio desde              int64
URL Nexoinmobiliario     object
dtype: object

### G. Guardar en google sheets 
drive-sheets-service@molten-topic-396921.iam.gserviceaccount.com


In [125]:
# 1. Crear flujo de lectura y escritura
sh = sheets.autentificar(os.path.join(os.getcwd(),'utils', 'keys_sheets.json'), '1PDphDA-IHCnuykZoDWjzkl_f0AxOsw0bUsIbpE3weZg','id')

# 2. Guardar df_proyectos_format en la hoja 'Proyectos'
sheets.dataframe_to_sheet(sh, df_proyectos_format, 'Proyectos')

# 3. Guardar df_modelos_format en la hoja 'Detalles'
sheets.dataframe_to_sheet(sh, df_modelos_format, 'Detalles')

In [126]:
print('Se guardaron los datos en Google Sheets')

Se guardaron los datos en Google Sheets
